### Simple option to generate parameter files

In [2]:
import numpy as np
import xarray as xr
import pandas as pd
import os
import glob

In [3]:
# Load file with parameter ranges
csv='ctsm53010_tests.txt'
params = pd.read_csv(csv)

In [7]:
# where do you want to save the parameter files?
out_dir = '/glade/work/linnia/CLM6-PPE/ctsm53010_tests/paramfiles/'

In [8]:
# default parameter file to start with:
basefile = '/glade/work/linnia/ctsm5.3.010/cime/scripts/transient/runtime_files/ctsm5.3.010_params_default_241111.nc'
def_paramfile = xr.open_dataset(basefile)

In [10]:
# Modify parameter settings and write new parameter values
for ens in range(len(params)):
    new_paramfile = def_paramfile.copy()
    param = params['param'][ens]
    values = params['value'][ens]
    float_list = [float(value) for value in values.split(',')]
    if len(float_list)>1: # PFT indexed
        array = def_paramfile[param].values
        array[:len(float_list)] = float_list[0:len(float_list)]
        print(param,array)
        if (param=='kmax'):
            new_paramfile['kmax'].isel(segment=3).data = array
        elif (param=='krmax'):
            new_paramfile['kmax'].isel(segment=4).data = array
        else:
            new_paramfile[param].data = array
    else:
        print(param,values)
        new_paramfile[param].data = values

    new_paramfile.to_netcdf(out_dir+params['key'][ens]+'.nc')
    

xl [0.   0.17 0.17 0.17 0.45 0.45 0.84 0.95 0.95 0.01 0.25 0.25 0.54 0.04
 0.04 0.54 0.54 0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3
 0.3  0.3  0.3  0.3  0.3  0.   0.   0.   0.   0.   0.   0.   0.   0.3
 0.3  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.3  0.3  0.   0.   0.   0.   0.3  0.3  0.
 0.   0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3 ]
xl [0.   0.17 0.17 0.17 0.45 0.45 0.84 0.95 0.95 0.01 0.25 0.25 0.54 0.04
 0.04 0.54 0.54 0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3
 0.3  0.3  0.3  0.3  0.3  0.   0.   0.   0.   0.   0.   0.   0.   0.3
 0.3  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.3  0.3  0.   0.   0.   0.   0.3  0.3  0.
 0.   0.3  0.3  0.3  0.3  0.3  0.3  0.3  0.3 ]
